Text Analytics I HWS 23/24

# Home Assignment 2 (26 pts)

Submit your solution via Ilias until 23.59h on Wednesday, October 25th. Late submissions are accepted until 12:00am on the following day, with 1/4 of the total possible points deducted from the score.

Submit your solutions in teams of 3-4 students. Unless explicitly agreed otherwise in advance, **submissions from teams with more or less members will NOT be graded**.
List all members of the team with their student ID in the cell below, and submit only one notebook per team. Only submit a notebook, do not submit the dataset(s) you used. Also, do NOT compress/zip your submission!

You may use the code from the exercises and basic functionalities that are explained in official documentation of Python packages without citing, __all other sources must be cited__. In case of plagiarism (copying solutions from other teams or from the internet) ALL team members may be expelled from the course without warning.

#### General guidelines:
* Make sure that your code is executable, any task for which the code does not directly run on our machine will be graded with 0 points.
* If you use packages that are not available on the default or conda-forge channel, list them below. Also add a link to installation instructions. 
* Ensure that the notebook does not rely on the current notebook or system state!
  * Use `Kernel --> Restart & Run All` to see if you are using any definitions, variables etc. that 
    are not in scope anymore.
  * Do not rename any of the datasets you use, and load it from the same directory that your ipynb-notebook is located in, i.e., your working directory.
* Make sure you clean up your code before submission, e.g., properly align your code, and delete every line of code that you do not need anymore, even if you may have experimented with it. Minimize usage of global variables. Avoid reusing variable names multiple times!
* Ensure your code/notebook terminates in reasonable time.
* Feel free to use comments in the code. While we do not require them to get full marks, they may help us in case your code has minor errors.
* For questions that require a textual answer, please do not write the answer as a comment in a code cell, but in a Markdown cell below the code. Always remember to provide sufficient justification for all answers.
* You may create as many additional cells as you want, just make sure that the solutions to the individual tasks can be found near the corresponding assignment.
* If you have any general question regarding the understanding of some task, do not hesitate to post in the student forum in Ilias, so we can clear up such questions for all students in the course.

In [1]:
# studentIDs of all team members
team_members = [1966868,1967897, 1968154, 1978986, 1951865  ]

Additional packages (if any):
 - Example: `powerlaw`, https://github.com/jeffalstott/powerlaw

In [2]:
from typing import List, Union, Dict, Set, Tuple, Sequence

### Task 1: WordNet word similarity (9 points)

In this task, we want to implement the similarity between two words in WordNet (https://www.nltk.org/api/nltk.corpus.reader.wordnet.html) using the NLTK package. The word similarity between two words is given by
$$
\frac{1}{1+d}
$$
where $d$ is the distance of the shortest path in the hypernym/hyponym hierarchy tree in WordNet between any pair of synsets that are associated with the two input words.

From NLTK you should __only__ use the `synsets`, `hypernyms` and `instance_hpyernyms` functions.

The following subtasks build on each other, i.e. the functions of the preceding subtasks can be used for the current subtask.

_Note: the distance of a synset to itself is 0, the distance to a direct hypernym is 1, ..._

In [3]:
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Synset

__a)__ Write a function ``shortest_paths_to`` that takes a synset as input and computes the shortest paths to all nodes on the way to the root in the hypernym hierarchy tree of WordNet. The function should return a dictionary that matches all visited hypernyms on the way(s) to the root to the distance of the shortest path from the input synset. Consider that a synset might have multiple paths to the root and that some nodes might appear in multiple paths. However, we only want to store the shortest distances. Moreover, keep in mind that the input synset might be an instance. __(3 pts)__

Use the signature in the cell below.

__Example:__ _Calling_ ``shortest_paths_to(s)`` _on the synset_ ``s = wn.synset('calculator.n.01')`` _should yield the following result:_

``
{Synset('calculator.n.01'): 0,
 Synset('expert.n.01'): 1,
 Synset('person.n.01'): 2,
 Synset('causal_agent.n.01'): 3,
 Synset('organism.n.01'): 3,
 Synset('physical_entity.n.01'): 4,
 Synset('living_thing.n.01'): 4,
 Synset('entity.n.01'): 5,
 Synset('whole.n.02'): 5,
 Synset('object.n.01'): 6}
``

In [4]:
def shortest_paths_to(start_syn: Synset) -> Dict[Synset, int]:
    """Compute the shortest distance to all nodes on paths to the root.
    :param start_syn: synset to which we want to compute the shortest distances
    :return: dict that matches all visited hypernyms to their distance to the input synset  
    """ 
    # Initialize a queue for BFS
    queue = [start_syn]
    # Initialize a dictionary to store distances
    distances = {start_syn: 0}
    
    # BFS Algorithm
    while queue:
        # Get the next synset from the queue
        current_syn = queue.pop(0)
        
        # Find hypernyms of the current synset
        hypernyms = current_syn.hypernyms()
        
        for hypernym in hypernyms:
            # whether the hypernym has been visited or processed
            if hypernym not in distances:
                # If this hypernym has not been visited before, add it to the queue
                queue.append(hypernym)
                # Compute the distance from the current synset to this hypernym
                distances[hypernym] = distances[current_syn] + 1
    
    return distances

shortest_paths_to(wn.synset('dog.n.01'))

{Synset('dog.n.01'): 0,
 Synset('canine.n.02'): 1,
 Synset('domestic_animal.n.01'): 1,
 Synset('carnivore.n.01'): 2,
 Synset('animal.n.01'): 2,
 Synset('placental.n.01'): 3,
 Synset('organism.n.01'): 3,
 Synset('mammal.n.01'): 4,
 Synset('living_thing.n.01'): 4,
 Synset('vertebrate.n.01'): 5,
 Synset('whole.n.02'): 5,
 Synset('chordate.n.01'): 6,
 Synset('object.n.01'): 6,
 Synset('physical_entity.n.01'): 7,
 Synset('entity.n.01'): 8}

__b)__ Write a function ``merge_paths`` that gets two dictionaries that map synsets to shortest distances (you can assume they were created by the function from __a)__) and merges them. The function shold return a dictionary that includes all synsets and distances that appear in any of the input dictionaries. If a synset appears in both input dictionaries, we want to keep the shorter distance. Leave the input dictionaries unaltered. __(1.5 pts)__

Use the signature in the cell below.

In [5]:
def merge_paths(p1: Dict[Synset, int], p2: Dict[Synset, int]) -> Dict[Synset, int]:
    """Merge two paths keeping the shorter distance for synsets that appear more than once.
    :param p1: first dict that maps synsets to their shortest distances
    :param p2: second dict that maps synsets to their shortest distances
    :return: merged dict
    """
    merged = p1
    for key in p2:
        if key not in merged:
            merged[key] = p2[key]
        elif merged[key] > p2[key]:
            merged[key] = p2[key]
    return merged

__c)__ Write a function ``all_hypernym_paths`` that gets a word as input and returns a dictionary that maps all hypernyms that are reachable from the set of synsets associated with the word to the shortest distance leading there. __(1.5 pts)__

Use the signature in the cell below.

In [6]:
def all_hypernym_paths(word: str) -> Dict[Synset, int]:
    """Get all hypernyms of all synsets associated with the input word and compute the shortest distance leading there.
    :param word: input word
    :return: dict that matches all reachable hypernyms to their shortest distance 
    """    
    synsets = wn.synsets(word)  # Get all synsets associated with the input word
    hypernym_paths = {}  # Initialize a dictionary to store hypernym paths

    for synset in synsets:
        # Compute the shortest paths to hypernyms for each synset
        synset_paths = shortest_paths_to(synset)
        
        # Merge the computed paths into the hypernym_paths dictionary
        hypernym_paths = merge_paths(hypernym_paths, synset_paths)

    return hypernym_paths

__d)__  Write a function ``get_dist`` that returns the word similarity between two input words, according to the formula given in the task description at the beginning.  __(3 pts)__

Use the signature in the cell below.

In [7]:
def get_dist(w1 : str, w2 : str) -> float:
    """Compute the similarity between two input words in the WordNet hierarchy tree.
    :param w1: first input word
    :param w2: second input word
    :return: word similarity
    """
    all_paths_w1 = all_hypernym_paths(w1)
    all_paths_w2 = all_hypernym_paths(w2)
    common_hypernyms = set(all_paths_w1.keys()).intersection(set(all_paths_w2.keys()))
    if len(common_hypernyms) == 0:
        return 0.0
    else:
        return 1 / (1+min([all_paths_w1[hypernym] + all_paths_w2[hypernym] for hypernym in common_hypernyms]))
    

print(get_dist('dog', 'house'))

0.125


### Task 2: Lesk algorithm (4 points)

In this task we want to implement a simple version of the Lesk algorithm, a thesaurus-based method for word sense disambiguation. Given a target word $w$ and a context, the algorithm finds the word sense that is most fitting in the context. To achieve this, the Lesk algorithm computes the number of overlapping words between the context sentence and the definitions of the WordNet synsets, associated with $w$.

Write a function ``lesk`` that takes a word and a context string (e.g. a sentence) and returns the most fitting sense from the synsets associated with the word and the corresponding context overlap. The most fitting sense is the one whose definition shares the most words with the context string. Before matching tokens, make sure to 

* only include valid tokens (cf. HA 1, task 2a)
* remove stopwords
* only match stems of words (e.g. consider the ``PorterStemmer`` from ``nltk``)

When computing the context overlap, count each stemmed word only once, even if they occur multiple times. If there is no fitting synset, i.e. the context overlap between the context and the synset definitions is 0, return None instead.

Use the signature in the cell below.

In [8]:
# HA 1, task 2a)
from nltk.corpus.reader.util import StreamBackedCorpusView 
from nltk.corpus import stopwords
import re
import string

def get_valid_tokens(tokens: Union[List[str], StreamBackedCorpusView], remove_stopwords: bool=False) -> List[str]:
    """
    :param tokens: list of tokens that should be cleaned
    :param remove_stopwords: bool indicating if stopwords should be removed 
                             False by default
    :return: list of valid tokens
    """
    valid = []
    punct = string.punctuation
    stop = stopwords.words('english')
    digit = re.compile(r"\d+")
    
    for t in tokens:
        if t in punct:
            continue
        if remove_stopwords and t.lower() in stop:
            continue
        if re.fullmatch(digit, t):
            continue
        valid.append(t.lower())
    return valid

In [9]:
from nltk import PorterStemmer, word_tokenize

def lesk(word: str, context: str) -> (Synset, int):
    '''
    Compute the most probable sense of a word in the given context.
    :param word: ambiguous word
    :param context: context in which the word appears
    :returns: 
        - synset with the most likely word sense
        - context overlap of synset and context          
    '''
    
    # Tokenize and stem the context
    context_tokens = get_valid_tokens(context.split(), remove_stopwords=True)
    stemmer = PorterStemmer() # reduce words to their base or root form
    context_stems = [stemmer.stem(token) for token in context_tokens]

    # Find synsets for the word
    synsets = wn.synsets(word)

    if not synsets:
        return None  # No fitting synsets, return None

    # Initialize variables to store the best sense and its overlap
    best_sense = None
    best_overlap = 0

    # Iterate through the synsets and find the best sense
    for synset in synsets:
        # Tokenize and stem the definition of the synset
        definition = synset.definition()
        definition_tokens = get_valid_tokens(word_tokenize(definition), remove_stopwords=True)
        definition_stems = [stemmer.stem(token) for token in definition_tokens]

        # Calculate the context overlap between context and synset definition
        overlap = len(set(context_stems).intersection(definition_stems))

        if overlap > best_overlap:
            best_sense = synset
            best_overlap = overlap

    return best_sense, best_overlap

In [10]:
synset, overlap = lesk("bank", "I went to the bank to deposit my money")

print(synset)
if synset:
    print(synset.definition())
print(overlap)

Synset('depository_financial_institution.n.01')
a financial institution that accepts deposits and channels the money into lending activities
2


### Task 3: Markov chains (13 points)

In this task we want to create a language model by using the independence assumption af Markov. We therefore assume that the probability of a word is only dependent on a fixed number of preceding words. E.g. by restricting the number of preceding words to $n$ we can approximate the probability of a word $w_{i}$ following a sequence of words $w_1, ..., w_{i-1}$ by:

$P(w_{i}|w_1, ..., w_{i-1}) \approx P(w_{i}|w_{i-n}, ..., w_{i-1})$

We will first train our model on a given corpus and then use it to automatically generate text.

Throughout this task we will define a single class with different functions. If you're unsure how to access class methods and attributes, take a look at the documentation (https://docs.python.org/3/tutorial/classes.html).

__a) Collecting the counts (3 pts)__

Write a function `process_corpus` that takes a corpus of text (as a sequence of tokens) as input and counts how often an n-gram of length $n$ (``context_len=n``) is followed by a certain word (the n-grams should __not__ be padded). The function should return a dictionary that maps every n-gram that is observed in the corpus to an inner dictionary. The inner dictionary maps each word to a number, that indicates how often the word succeeds the n-gram in the given corpus. We will need these counts to compute the conditional probabilities $P(w_{i}|w_{i-n}, ..., w_{i-1})$.
Additionally, also return the entire vocabulary $V$ (i.e. the set of all unique tokens that appear in the corpus).

Make sure your implementation is efficient by using e.g. ``Counter`` and ``defaultdict`` from the package ``collections``.   

__b) Conditional probabilities (3 pts)__

Write a function `transition_prob` that takes an n-gram $(w_{i-n}, ..., w_{i-1})$ and a word $w_{i}$ of the vocabulary $V$ as input and returns the conditional probability that the given n-gram is followed by the input word $w_{i}$. Recall that this conditional probability can be computed as follows:

$P(w_{i}|w_{i-n}, ..., w_{i-1}) = \frac{\text{Count}(w_{i-n}, ..., w_{i-1}, w_{i})}{\sum_{w \in V}\text{Count}(w_{i-n}, ..., w_{i-1}, w)}$

If the n-gram has never been observed, return $\frac{1}{|V|}$.

__c) Most likely word (3 pts)__

Write a function `most_likely_word` that gets an n-gram as input and returns the word that is most likely to succeed the given n-gram. In case there are multiple words that are equally likely to follow the given n-gram, return all of them. 
Note that you should **not** loop over the **entire** vocabulary to obtain the most likely word.
In case the given n-gram has never been observed, return the entire vocabulary.

__d) Generating text (2 pts)__

Write a function `generate_text` that generates a text sequence of length `k`, given a starting sequence of words (`ngram`). The function should choose the most likely next word in every step; in case there are multiple equally likely words, randomly choose one. You should return a list of ``k`` words, that includes the starting sequence and is the most probable continuation. 


Please do not implement other functions for the MarkovModel class.

Use the function signatures in the cell below.

In [11]:
from collections import defaultdict, Counter
import random
from nltk.util import ngrams

class MarkovModel():
    '''Markov model for generating text.'''
    
    def __init__(self, tokens: Sequence[str], context_len: int):
        '''
        :param tokens: text corpus on which the model is trained on as an iterator of tokens
        :param context_len: length of the n-gram (number of preceding words)
        '''
        self.context_len = context_len 
        self.counts, self.v = self.process_corpus(tokens)
        
    def process_corpus(self, tokens: Sequence[str]) -> (Dict[Tuple[str, ...], Dict[str, int]], Set):
        '''Training method of the model, counts the occurrences of each word after each observed n-gram.
        :param tokens: text corpus on which the model is trained on as an iterator of tokens
        :returns: 
            - nested dict that maps each n-gram to the counts of the words succeeding it
            - the whole vocabulary as a set
        '''
        # Initialize a dictionary to store n-gram counts and a set for the vocabulary.
        ngram_counts = defaultdict(Counter)
        vocabulary = set(tokens)
        
        # Iterate through the corpus using ngrams, catching following word
        for context_ngram in ngrams(tokens, self.context_len + 1):
            # Split the n-gram into a context (preceding words) and the current word.
            ngram = tuple(context_ngram[:-1])
            word = context_ngram[-1]
            
            # Update the n-gram counts by incrementing the count of the current word following the context.
            ngram_counts[ngram][word] += 1
        
        # Return the n-gram counts and vocabulary set.
        return ngram_counts, vocabulary
    
        
    def transition_prob(self, ngram: Tuple[str, ...], word: str) -> float:
        '''Compute the conditional probability that the input word follows the given n-gram.
        :param ngram: string tuple that represents an n-gram
        :param word: input word
        :return: probability that the n-gram is followed by the input word
        '''
        if ngram in self.counts:
            ngram_counts = self.counts[ngram]
            total_counts = sum(ngram_counts.values())
            return ngram_counts[word] / total_counts
        return 1.0 / len(self.v)
    
    def most_likely_word(self, ngram: Tuple[str, ...]) -> Set[str]:
        '''Computes which word is most likely to follow a given n-gram.
        :param ngram: n-gram we are interested in
        return: set of words that are most likely to follow the n-gram
        '''
        if ngram not in self.counts:
            return self.v
        return set(k for k,v in self.counts[ngram].most_common())
    
    def generate_text(self, ngram: Tuple[str, ...], k: int) -> List[str]:
        '''Generates a text sequence of length k, given a starting sequence.
        :param ngram: starting sequence
        :param k: total number of words in the generated sequence
        :return: sequence of generated words, including the starting sequence
        '''
        # Initialize the generated text with the provided starting n-gram
        generated_text = list(ngram)

        # Generate the remaining words in the sequence
        for _ in range(k - len(ngram)):
            if ngram in self.counts:
                # If the current n-gram is observed, use most_likely_word to get the most likely next word
                most_likely_words = self.most_likely_word(ngram)
                next_word = random.choice(list(most_likely_words))
            else:
                # If the current n-gram is not observed, select a random word from the vocabulary
                next_word = random.choice(list(self.v))

            # Append the selected next word to the generated text
            generated_text.append(next_word)

            # Update the n-gram by shifting one word forward
            ngram = tuple(list(ngram)[1:] + [next_word])

        return generated_text

__e) Apply the model to a corpus (2 pts)__

Finally, we want to apply our functions to the King James Bible (`'bible-kjv.txt'`) that is part of the ``gutenberg`` corpus. Use the function from HA 1, task 2a) to obtain a list of valid tokens (do not remove stopwords) from the King Jame Bible.

Initialize the MarkovModel with the list of valid tokens and ``context_len=3`` and answer the following subtasks:

i) What is the probability that the word ``babylon`` follows the sequence ``the king of``? 

ii) What are the most likely words to follow the sequence ``the world is``? 

iii) Generate a sequence of length 20 that starts with ``mother mary was``.


In [12]:
import nltk
king_james_bible = nltk.corpus.gutenberg.raw('bible-kjv.txt')
king_james_bible_tokens = nltk.word_tokenize(king_james_bible)

valid_tokens = get_valid_tokens(king_james_bible_tokens)
model = MarkovModel(valid_tokens, 3)

babylon_prob = model.transition_prob(('the', 'king', 'of'), 'babylon')
print("The probability of seing 'babylon' after 'the king of' is: ", babylon_prob)

the_world_is = model.most_likely_word(('the', 'world', 'is'))
is_are = "is: "+the_world_is[0] if len(the_world_is) == 1 else "are: " + str(the_world_is)
print("The most likely word to follow 'the world is'", is_are )

generated = model.generate_text(('mother', 'mary', 'was'),20)
print("Sentance of length 20 starting with 'Mother mary was':")
print(generated[0][0].upper()+generated[0][1:]+' '+' '.join(generated[1:])+'.')

The probability of seing 'babylon' after 'the king of' is:  0.18723404255319148
The most likely word to follow 'the world is' are: {'mine', 'the', 'enmity', 'gone', 'crucified'}
Sentance of length 20 starting with 'Mother mary was':
Mother mary was espoused to joseph before they came together she was found with child of gilead that they might.
